## POLSCI 3  Lecture Notebook: Hypothesis Testing

In this notebook we will learn how to evaluate hypotheses using R. We begin with tabular analysis (2 categorical variables), proceed to difference in means (1 categorical variable, 1 continuous variable), and end with the correlation coefficient (2 continuous variables).

### Tabular Analysis  
In the following example, we will explore how to determine if there is a relationship between two categorical variables. We start with the example from our textbook, on the relationship between gender and vote choice in the 2016 US presidential election. The table below contains a sample of 2440 voters who were asked their gender and vote choice in the election. Cell values are the number of respondents in the corresponding category. (See also the textbook, p. 170.) *Note that this table excludes the row and column totals. You do not need to understand how to create a table like this*.

In [ ]:
respond_table <- as.table(cbind(c(532, 596),c (737, 575)))
dimnames(respond_table) <- list(candidate = c("Clinton","Trump"),
                                gender = c("Male","Female")
                                )
respond_table

In the introductory lecture on tabular analysis we actually went through all the intermediate steps to determine whether there was a relationship:

1. We computed the expected values of each cell under the null hypothesis.
2. For the table we computed $$ \chi^2 = \sum \frac{(Observed-Expected)^2}{Expected}$$

3. We compared the computed value of $\chi^2$ to the corresponding critical value to assess
   whether we can reject the null hypothesis of no relationship.

The nice thing about working in R is that R performs does all these computations for us in one command:

In [ ]:
chisq.test(respond_table)

Note the output of the `chisq.test` command gives not just the value of $\chi^2$ but the degrees of freedom (df) and the p-value too. 

Recall that **a _p_-value is the probability of seeing an estimate as large or larger than the estimate we saw _if there was no relationship in the population_**. Note above that the p-value is super small, on the order of .00001

Another way of thinking about what a _p_-value tells us: if we drew repeated random samples and computed the $\chi^2$ for each one, what proportion of the time would we get an estimate this large if there were no relationship in the population (the null hypothesis)? In our example above it would be 1 in 100,000. 

**Social scientists have arbitrarily decided to denote all estimates with *p*-values below 0.05 statistically significant**. That means we have arbitrarily decided that, if there is a 1 in 20 chance or less that we would get the sample estimate we did if there were not relationship in the population, we decide to conclude for the sake of scientific discussions that there is a relationship in the population. (For reference, in physics they require _p_-values of 1 in 3.5 million.)

## Difference of Means

When we have a continuous variable categorized by a categorical variable, we can use a *difference of means* test to determine if there is a relationship between the categorical variable and the continuous variable. For example, we would use a difference of means test to determine if there was a relationship between ice cream flavor (a categorical variable) and sales revenue (a continuous variable). 

In the example below, we follow the textbook (pp. 173-178) and analyze whether there is a significant difference in duration of governance between majority and minority governments. Our ex ante expectation is that majority governments last longer than minority governments. 

First let's load in the data:

In [ ]:
govts<-read.csv('govts.csv')

Here is the codebook that describes the variables:

`nation`: the name of the country

`govtdate`: the date the government takes power

`govttime`: the amount of time in days the government is in control

`mingov`: a binary categorical variable where 1 signifies a minority government and 0 a majority government

In [ ]:
head(govts)

We need to plot the distributions of <code>govttime</code> for minority and majority governments. 
Let's start by creating two subsets, one for minority governments and one for majority governments. 

In [ ]:
min<-subset(govts,mingov==1)
maj<-subset(govts,mingov==0)

Now let's plot the densities of <code>govttime</code> for each type of government. (Compare to Figure 8.2 in the textbook.)

In [ ]:
plot(density(min$govttime), col = "blue", xlim = c(0, 2000))
lines(density(maj$govttime), col = "red")

Are the mean durations for majority and minority governments sufficiently different from one another to justify rejecting the null hypothesis of no difference? In the difference of means lecture we computed the standard errors, t-statistic, and degrees of freedom by hand. The advantage of R is that R has a built in function for a difference of means test:

In [ ]:
t.test(maj$govttime, min$govttime)

Recall that a t-statistic is the ratio of the estimate (sanple difference in means) to the standard error:

$t = \frac{\text{Estimate}}{\text{Standard Error}}$

**The _t_-statistic is a way to measure how likely it is that an estimate of the size we saw would arise by chance even if in the population there was no difference in means.** The larger the t-value, the smaller the p-value. 

By convention, **we call an estimate _statistically significant_ if the _t_-statistic is larger than 1.96 (either lower than -1.96 or larger than 1.96).** Smaller than that (closer to zero), and we generally conclude that our estimate could have arisen by chance (and we do not reject the null hypothesis).

## Pearson's r: Continuous Variables
When both variables are continuous, we want to measure their *correlation*: the degree to which one of the variable's change in value coincides with a similar change in value in the other variable. In the following example, we return to our earlier dataset on elections and explore the correlation between GDP and incumbent vote share. We expect this correlation to be positive. That is, as GDP growth increases, we expect incumbent vote share to increase also.  

In [ ]:
elections <- read.csv('FairFPSR3.csv')
head(elections)

Here is the codebook:

`inc_vote`: % of major party presidential vote won by incumbent party

`year`: Year of the presidential election

`inflation`: Inflation rate

`goodnews`: Number of quarters in the first 15 quarters of admin in which econ growth>3.2%

`growth`: % change in real GDP per capita

When computing correlations it is always a good idea to visualize the variables you are correlating by graphing them. (See also Figure 8.3 in the textbook.)

In [ ]:
plot(elections$growth,elections$inc_vote)   # First variable is X axis, second variable is Y axis

Is the relationship positive? negative? Or is there no relationship between the variables?

To calculate the population correlation coefficient in R, we use `cor.test(X,Y)` where X and Y are 
variables. Below, we calculate the correlation coefficient for growth and incumbent vote share:

In [ ]:
cor.test(elections$growth,elections$inc_vote)

Given the above output, what can we conclude about the relationsip between GDP growth and incumbent vote share?

## Interactive Visual 
The code in the following cell generates a distribution based on its correlation with the random uniform distirubtion x. This is meant to visualize what the effect of increasing or decreasing the correlation has on the visual relationship between variables. YOU DO NOT NEED TO UNDERSTAND HOW THIS CELL WORKS.

In [ ]:
x=runif(n=50, min=0, max=1)
simcor = function (x, correlation) { 
    n = length(x)
    ymean = 0
    ysd = 1
    y = rnorm(n) 
    z = correlation * scale(x)[,1] + sqrt(1 - correlation^2) * scale(resid(lm(y ~ x)))[,1] 
    yresult = ymean + ysd * z 
    yresult }
y=simcor(x,0.85)                                 # Try changing the 0.5 to something else between -1 and 1 
                                               # and reexecuting cell
plot(x,y)

#### Reminder about Peer Consulting Office Hours

If you had trouble with any content in this notebook, Data Peer Consultants are here to help! You 
can view their locations and availabilites at this link: https://data.berkeley.edu/degrees/peer-advising.
Peer Consultants are there to answer all data-related questions, whether it be about the content of this notebook,
applications of data science in the world or other data science courses offered at Berkeley -- 
make sure to take advantage of this wonderful resource!